# Notebook: Calculate Agreement

This notebook is used to calculate the inter-rater agreement using Krippendorf's Alpha.
<br>**Contributors:** [Nils Hellwig](https://github.com/NilsHellwig/) | [Markus Bink](https://github.com/MarkusBink/)

## Packages

In [1]:
from statsmodels.stats import inter_rater as irr
import krippendorff as kd
import pandas as pd
import numpy as np
import glob
import os

## Parameters

In [2]:
ANNOTATED_DATASET_PATH = "../Datasets/annotated_dataset/"
LABEL_CODING = {'NEUTRAL': 3, 'NEGATIVE': 2, 'POSITIVE': 1, 'MIXED': 4}

## Settings

In [3]:
pd.options.display.max_colwidth = 1000

## Code

### 1. Load Annotations

In [4]:
file_list = sorted(glob.glob(ANNOTATED_DATASET_PATH + "*.xlsx"))

In [5]:
# read and concatenate annotator 1's session 1 and 2 data
df_annotator_1 = pd.concat([
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_1.xlsx"),
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_1.xlsx")
])

In [6]:
# rename sentiment column and recode labels
df_annotator_1.rename(columns={'sentiment': 'sentiment_1'}, inplace=True)
df_annotator_1['sentiment_1'] = df_annotator_1['sentiment_1'].map(LABEL_CODING)

In [7]:
# read and concatenate annotator 2's session 1 and 2 data
df_annotator_2 = pd.concat([
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_2.xlsx"),
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_2.xlsx")
])

In [8]:
# rename sentiment column and recode labels
df_annotator_2.rename(columns={'sentiment': 'sentiment_2'}, inplace=True)
df_annotator_2['sentiment_2'] = df_annotator_2['sentiment_2'].map(LABEL_CODING)

In [9]:
# read and concatenate annotator 2's session 1 and 2 data
df_annotator_3 = pd.concat([
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_3.xlsx"),
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_3.xlsx")
])

In [10]:
# rename sentiment column and recode labels
df_annotator_3.rename(columns={'sentiment': 'sentiment_3'}, inplace=True)
df_annotator_3['sentiment_3'] = df_annotator_3['sentiment_3'].map(LABEL_CODING)

In [11]:
# concatenate annotator 1 and 2 data
df_all_annotations = pd.concat([
    df_annotator_1[['id','tweet','sentiment_1']], 
    df_annotator_2[['sentiment_2']],
    df_annotator_3[['sentiment_3']]
], axis=1)

In [12]:
# check for missing values in sentiment columns
print(df_all_annotations[df_all_annotations['sentiment_1'].isnull()])
print(df_all_annotations[df_all_annotations['sentiment_2'].isnull()])
print(df_all_annotations[df_all_annotations['sentiment_3'].isnull()])

Empty DataFrame
Columns: [id, tweet, sentiment_1, sentiment_2, sentiment_3]
Index: []
Empty DataFrame
Columns: [id, tweet, sentiment_1, sentiment_2, sentiment_3]
Index: []
Empty DataFrame
Columns: [id, tweet, sentiment_1, sentiment_2, sentiment_3]
Index: []


In [13]:
df_all_annotations = df_all_annotations.reset_index(drop=True)

In [14]:
df_all_annotations

,id,tweet,sentiment_1,sentiment_2,sentiment_3
0,1460589265759480064,"@FrankieLix2020 @andfra9 @GoeringEckardt @ABaerbock @OlafScholz @spdbt @GrueneBundestag @fdpbt @LieblingXhain @ninastahr Ist doch egal, was das RKI sagt, weil.... Ja, FDP, SPD &amp; Grüne, warum eigentlich? Wir Eltern warten immer noch auf eine Erklärung, warum ihr unsere Kinder durchseuchen wollt. Besonders schön wäre eine stichhaltige &amp; plausible Erklärung.",2,4,4
1,1470141403045019904,Alter! Der @Karl_Lauterbach ist ja schon wieder in einer Talkshow. Ich dachte es endet jetzt und er hat zutun! #karllauterbach #annewill,3,2,2
2,1405950770466497024,@Karl_Lauterbach Sie sehen auch so aus.,3,3,3
3,1350126525220314880,"@PaulZiemiak @CDU 2/x »Wenn die Bürger die Wahl hätten zwischen der parlamentarischen Demokratie, bei der die gewählten Abgeordneten Entscheidungen treffen und Gesetze beschließen und direkter Demokratie, bei der die Bürger in Sachfragen durch Volksabstimmungen entscheiden könnten, würden sich nur",3,3,3
4,1443988844500733952,@ArminLaschet Laschet ist als CDU-Parteivorsitzender peinlich. Ein neuer Parteivorsitzender der CDU muss gewählt werden. Laschet soll sofort zurücktreten! @CDU @CDUNRW_de @csu_lt @ArminLaschet @CSU @csu_bt @CDUNRW_de @CDU_CSU_EP @Wirtschaftsrat @CDU_BW @cdu_hessen @cdurlp @welt @FAZ_Politik,2,2,2
...,...,...,...,...,...
1995,1400720644279504896,"@spdde @OlafScholz mit einem Wumms aus der Krise. Sorry, das ist Volksverblödung",2,2,2
1996,1415980314871087104,@manager_magazin Wieviel kriegt @OlafScholz weil er die Konkurrenz und andere Firmen mit Steuer belegt?,3,3,2
1997,1419538832488374016,"@jensteutrine @fdp @c_lindner Immer und immer wird behauptet, Autos seien umweltschädlich und würden den Klimawandel vorantreiben. In Wahrheit ermöglichen Sie Mobilität unabhängig vom sozialen Hintergrund. Hierzu 5 E-Autos:",2,3,4
1998,1436026687548953088,"@Mirko17817058 @sebastiankurz @ArminLaschet Naja... Wir haben CumEx, Rechtswidrige Polizeieinsätze und Fehlerchen in einem Buch... wollen Sie das jetzt wirklich alles gleichwertig betrachten?",3,3,2


### 2. Get Examples for Paper

In [15]:
df_agreement = df_all_annotations.loc[(df_all_annotations['sentiment_1'] == df_all_annotations['sentiment_2']) & (df_all_annotations['sentiment_1'] == df_all_annotations['sentiment_3'])]

Show positive agreement

In [16]:
df_agreement[df_agreement["sentiment_1"] == 1][:5]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
20,1447576229176122880,"#Legalisierung von Hanf bedeutet eine neue Ehrlichkeit in der Drogenpolitik! @OlafScholz , @Larsklingbeil , @EskenSaskia , @KuehniKev , @c_lindner , @ABaerbock , @Wissing",1,1,1
28,1449152380054934016,@23rasm @VQuaschning @spdbt @Die_Gruenen @fdp Auf den Punkt gebracht.,1,1,1
57,1373752135331033088,@Karl_Lauterbach Meine Rückendeckung haben Sie! Machen Sie bitte weiter.,1,1,1
151,1370678455973646080,@AnkeHoffnung @Karl_Lauterbach @grimmsi1 @TOODALOO5 @Richter_Mueller @HeinoStoever @WurthGeorg @philineedbauer @NiemaMovassat @KirstenKappert @MAStrackZi @haucap @CDR_FFM @hanfverband Danke dir für die Liste ! 😉,1,1,1
175,1378325746121335040,"@CorneliusRoemer @CDU @AfD @ArminLaschet @infratestdimap Die Befürworter eines Lockdownssind bei den Grünen und bei der CDU, also bei Wählern, denen es gut geht, die viel verdienen. Vielleicht denkt Herr Laschet an die sozial Schwachen, die um ihre Existenz oder um die psychische Lage ihrer Kinder fürchten. Vorbildlich, finde ich.",1,1,1


Show negative agreement

In [17]:
df_agreement[df_agreement["sentiment_1"] == 2][:5]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
4,1443988844500733952,@ArminLaschet Laschet ist als CDU-Parteivorsitzender peinlich. Ein neuer Parteivorsitzender der CDU muss gewählt werden. Laschet soll sofort zurücktreten! @CDU @CDUNRW_de @csu_lt @ArminLaschet @CSU @csu_bt @CDUNRW_de @CDU_CSU_EP @Wirtschaftsrat @CDU_BW @cdu_hessen @cdurlp @welt @FAZ_Politik,2,2,2
5,1396429761682021888,.@ABaerbock @Die_Gruenen @cducsubt @spdbt @dielinke @fdpbt : So viel Wald vernichtet unser Konsum https://t.co/NzwU03kjXa #vegan #palmoilfree #vegan #cancelanimalag #Erdueberlastungstag,2,2,2
6,1382709511153189120,@Karl_Lauterbach auch die Lehrkraefte werden nicht geimpft. Geht gar nicht! #bildungabersicher,2,2,2
7,1347699309492494080,@afd_wallduern Mit keinem Wort fordert er dieses Sperren aber wieder typisch @AfD lügen verbreiten.,2,2,2
10,1390305016808828928,"@PDominke @AfDimBundestag Pauschale Verunglimpfung ist ein beliebtes Hobby, also kein Grund zur Sorge. Es gab zu allen Zeiten Gruppen, auf die man eingehackt hat im Gefühl der trauten Gemeinsamkeit der rechthaberischen Mehrheit. Ihnen fiel ja auch nichts Anderes zum Thema ein.",2,2,2


Show neutral agreement

In [18]:
df_agreement[df_agreement["sentiment_1"] == 3][:5]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
2,1405950770466497024,@Karl_Lauterbach Sie sehen auch so aus.,3,3,3
3,1350126525220314880,"@PaulZiemiak @CDU 2/x »Wenn die Bürger die Wahl hätten zwischen der parlamentarischen Demokratie, bei der die gewählten Abgeordneten Entscheidungen treffen und Gesetze beschließen und direkter Demokratie, bei der die Bürger in Sachfragen durch Volksabstimmungen entscheiden könnten, würden sich nur",3,3,3
9,1371730424196701952,@Schmidtlepp @jensspahn SpahnScheuer,3,3,3
14,1447496797295975936,"@nureinleser10 @M_van_Laack @Matthias_Kamann @Joerg_Meuthen Will die AfD irgendwann mal regieren oder wozu ist sie da? Sie oder andere sagen doch, dass es eine schwarz-gelb-blaue Mehrheit gibt, wieso werden dann keine Angebote gemacht? Wieso muss sich die CDU ausgerechnet an der AfD orientieren? Gibt doch auch FDP, Freie Wähler, CSU",3,3,3
15,1345337052557156096,@rRockxter @europeika @CDU Was hat denn die CDU mit dem Christentum zu tun? 🤔 https://t.co/ioFmt1Zny6,3,3,3


Show mixed agreement

In [19]:
df_agreement[df_agreement["sentiment_1"] == 4][:5]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
58,1436397401720372992,@Jim_na_Jim @toniturtle1963 @Karl_Lauterbach Die Maske trägt sich sehr gut. Aber die getackerten Bänder sind nicht optimal. Wenn man sie nicht vorsichtig auf- oder absetzt dann lösen sie sich.,4,4,4
62,1425341079730233088,"@paramotor_lutz @Luisamneubauer @ArminLaschet @OlafScholz Wäre ja auch dumm gewesen bei Atom zu bleiben. Das hätte uns nämlich rein vom Gefühl her noch mehr Zeit verschafft, nicht auf die erneuerbaren Energien umzusteigen. Verzögerung ist übrigens die beste Form der Sabotage. Also alles gut so.",4,4,4
90,1432302887892398080,"@HeidiSigrid @PN46PN46 @Markus_Soeder Ich weiß nicht woran es liebt, aber Habeck und Baerbock kommen einfach menschlicher rüber. Der Rest wirkt wie seelenlose Politiker.",4,4,4
140,1431878456376233984,"@_FriedrichMerz Danke, dass Sie jetzt schon zugeben, die Wahl zu verlieren. Ihr Einsehen kommt sehr früh. Hätte ich gerade von Ihnen nicht erwartet.",4,4,4
148,1445099445192904960,"@Markus_Soeder Ich setze auf Eigenverantwortung, Freiheit und Sicherheit. #btw21 Der Untergang der CSU bei der Wahl sollte eigentlich eine Erleuchtung bringen",4,4,4


Show no agreement

In [20]:
df_all_annotations[(df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_2"]) &
             (df_all_annotations["sentiment_2"] != df_all_annotations["sentiment_3"]) &
             (df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_3"])][:5]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
1004,1436969755689299968,"@jensspahn @MikeMohring Mein Ort hat nur 200 EW und hier fährt 2x am Tag der Schulbus, mehr ÖPNV ist hier nicht. ABER: ich wähle grün, weil vor allem die CDU dazu beigetragen hat, dass wir hier nicht ohne ein 2. Auto auskommen. Wir wählen bessere Mobilität, und geben dann gerne ein Auto ab.",1,3,4
1140,1384467490441474048,@ArminLaschet wurd sowieso nie Kanzler weil jetzt die @SPD gewinnen wird!! @NRW_Ministerium @CDU @cducsubt @CSU,2,1,3
1193,1449065935785139968,@gb_1960 @SWagenknecht Die Gehirnwäsche hat gewirkt. Du hättest herrlich in die DDR gepasst.,4,2,3
1333,1361475742127836928,"@SHomburg @Karl_Lauterbach @BrinkmannLab Herr Homburg, haben Sie Sympathie für Hass?",2,4,3
1366,1464296164728226048,"@Karl_Lauterbach was n fuchs der karl :) mal schnell n inhaltsverzeichnis studiert und tweet gesetzt. wahrscheinlich, sicherlich, aufjedenfall! wirkt! eventuell! weil gefährlich! aber booster schützt! muss booster! 3 Monate.... oh mann",2,3,4


### 3. Sentiment Class Distribution

n positive majority

In [21]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 1) & (df_all_annotations["sentiment_2"] == 1)) |
                    ((df_all_annotations["sentiment_2"] == 1) & (df_all_annotations["sentiment_3"] == 1)) |
                    ((df_all_annotations["sentiment_1"] == 1) & (df_all_annotations["sentiment_3"] == 1)))])

128

n negative majority

In [22]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 2) & (df_all_annotations["sentiment_2"] == 2)) |
                       ((df_all_annotations["sentiment_2"] == 2) & (df_all_annotations["sentiment_3"] == 2)) |
                       ((df_all_annotations["sentiment_1"] == 2) & (df_all_annotations["sentiment_3"] == 2)))])

924

n neutral majority

In [23]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 3) & (df_all_annotations["sentiment_2"] == 3)) |
                       ((df_all_annotations["sentiment_2"] == 3) & (df_all_annotations["sentiment_3"] == 3)) |
                       ((df_all_annotations["sentiment_1"] == 3) & (df_all_annotations["sentiment_3"] == 3)))])

815

n mixed majority

In [24]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 4) & (df_all_annotations["sentiment_2"] == 4)) |
                       ((df_all_annotations["sentiment_2"] == 4) & (df_all_annotations["sentiment_3"] == 4)) |
                       ((df_all_annotations["sentiment_1"] == 4) & (df_all_annotations["sentiment_3"] == 4)))])

109

n no majority

In [25]:
len(df_all_annotations[(df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_2"]) &
             (df_all_annotations["sentiment_2"] != df_all_annotations["sentiment_3"]) &
             (df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_3"])])

24

### 4. Calculate Krippendorff's Alpha

In [26]:
# Rows are the coders (annotators) # of coders
# Columns are the individual items (sentiment of tweet) # of tweets
value_counts = df_all_annotations[["sentiment_1", "sentiment_2", "sentiment_3"]]
value_counts = value_counts.to_numpy().transpose()
kd.alpha(reliability_data=value_counts, level_of_measurement="nominal")

0.7247970085578127

### 5. Calculate Fleiss' Kappa

In [27]:
agg = irr.aggregate_raters(df_all_annotations[["sentiment_1", "sentiment_2", "sentiment_3"]])

In [28]:
irr.fleiss_kappa(agg[0], method='fleiss')

0.7247511337467707